In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Read in the processed tumor volume data:
df = pd.read_csv('processed_volumes_3.csv')
print(df.shape)

# # Split name cases
df['Case.Patient'] = df['patient_id'].apply(lambda x: x.split('\\')[-1])

# Ensure scan_date column is converted to datetime format
df['scan_date'] = pd.to_datetime(df['scan_date'], format='%m/%d/%Y')

# Now you can use the .dt accessor to format the dates
df['Case.Date'] = df['scan_date'].dt.strftime('%Y-%m-%d')

df['Case.Scan'] = df['case_number'].apply(lambda x: '_'.join(x.split('_')[2:]))

df = df.drop(columns=['case_number', 'phase', 'patient_id','scan_date', 'tumor_surface_area',
                       'tumor_sphericity',  'tumor_volume_to_area_ratio', 'dicom_center_of_mass_x', 'dicom_center_of_mass_y', 'dicom_center_of_mass_z'])
df = df.rename(columns={'tumor_volume': 'Volume', 'patient_center_of_mass_x': 'CoordinateSagittal', 'patient_center_of_mass_y': 'CoordinateCoronal', 'patient_center_of_mass_z':'CoordinateTransverse'})

# # Adapt volume to mL instead of mm^3
df['Volume'] = df['Volume'] / 1000

# # Remove small volumes below 1mL
df = df.loc[df['Volume'] > 1.]
volume_df = df
df

In [ ]:
# Group by patient_id and find the last check-up date for each patient
last_check_up_dates = volume_df.groupby('Case.Patient')['Case.Date'].max().reset_index()
last_check_up_dates

## Total Volume Analysis


In [ ]:
import numpy as np

# We dont need the coordinates at this stage, so we can remove them safely
df = df.drop(columns=['CoordinateSagittal', 'CoordinateCoronal', 'CoordinateTransverse'])

# Compute total volume and range for each patient and date
df = df.groupby(['Case.Patient', 'Case.Date', 'Case.Scan']).sum().reset_index()
df = df.drop(columns='Case.Scan')
df = df.groupby(['Case.Patient', 'Case.Date']).agg(
    Volume_Mean=('Volume', 'mean'),
    Volume_Range=('Volume', lambda x: np.nan if x.size == 1 else x.max() - x.min())
).reset_index()
df = df.rename(columns={'Volume_Mean': 'Volume.Mean', 'Volume_Range': 'Volume.Range'})

df

## Build an interval dataset


In [ ]:
# Convert Case.Date to datetime
df['Case.Date'] = pd.to_datetime(df['Case.Date'])

# Sort by Case.Patient and Case.Date
df = df.sort_values(by=['Case.Patient', 'Case.Date'])

# Create shifted columns for pairs
df['Case.Date.End'] = df.groupby('Case.Patient')['Case.Date'].shift(-1)
df['Volume.Mean.End'] = df.groupby('Case.Patient')['Volume.Mean'].shift(-1)
df['Volume.Range.End'] = df.groupby('Case.Patient')['Volume.Range'].shift(-1)

# Rename original columns for clarity
df.rename(columns={
    'Case.Date': 'Case.Date.Start',
    'Volume.Mean': 'Volume.Mean.Start',
    'Volume.Range': 'Volume.Range.Start'
}, inplace=True)

# Drop rows where Case.Date.End is NaN (the last entry for each patient)
df = df.dropna(subset=['Case.Date.End'])

# Select and reorder relevant columns
df = df[['Case.Patient', 'Case.Date.Start', 'Case.Date.End', 'Volume.Mean.Start', 'Volume.Mean.End', 'Volume.Range.Start', 'Volume.Range.End']]

df

## Load treatment data

In [ ]:
# load treatment data
treatment = pd.read_excel('W_23_5814_2024-03-06.xlsx', sheet_name='Oncolytica')
print('Oncolytica shape: ', treatment.shape)
# filter on the treatment of interest
treatment = treatment.loc[treatment['ATC_CODE'].apply(lambda x: str(x).startswith('L01') or str(x).startswith('L02'))]
treatment = treatment.loc[treatment['ISLAATST']]
treatment = treatment.loc[treatment['DOSERING'].apply(lambda x:'per dag 0' not in str(x))]


# select columns of interest 
treatment = treatment[['PATIENTNR', 'VOORSCHRIJFDATUM', 'VOORSCHRIJFEINDDATUM']]
treatment.rename(columns={'PATIENTNR': 'Case.Patient', 'VOORSCHRIJFDATUM': 'Treatment.Start', 'VOORSCHRIJFEINDDATUM': 'Treatment.End'}, inplace=True)

# laod surgery data
surgery = pd.read_excel('W_23_5814_2024-03-06.xlsx', sheet_name='Surgery')
print('surgery shape: ', surgery.shape)

# filter on the ones for the liver: *lever*, *embolisatie* 
surgery = surgery.loc[surgery['OKVR_Verrichting_omschrijving'].apply(lambda x: 'lever' in str(x).lower() or 'embolisatie' in str(x).lower() or 'rfa' in str(x).lower())]

# select columns of interest
surgery = surgery[['PATIENTNR', 'OK_Operatiedatum']]
surgery.rename(columns={'PATIENTNR': 'Case.Patient', 'OK_Operatiedatum': 'Treatment.Start'}, inplace=True)
surgery['Treatment.End'] = surgery['Treatment.Start']

# merge treatment and surgery data as intervention data
# surgery date becomes the start and end of the intervention
intervention = pd.concat([treatment, surgery], axis=0, ignore_index=True).reset_index(drop=True)

# load patient ID keys for anonymization
keys = pd.read_csv('20230926 JADS Export Report Patients.csv')
print('keys shape: ', keys.shape)

# use anonymization keys to replace patient IDs
keys = keys.loc[~(keys['Patient ID'] == '20401820xxx187484')]
keys['Patient ID'] = keys['Patient ID'].astype(int)

intervention = pd.merge(intervention, keys, left_on='Case.Patient', right_on='Patient ID', how='left')
intervention = intervention.drop(columns=['Case.Patient', 'Patient ID'])
intervention.rename(columns={'Anonymized Name': 'Case.Patient'}, inplace=True)

intervention

## Add treatment interval indicator

In [ ]:
# Convert Case.Date.Start and Case.Date.End to datetime
df['Case.Date.Start'] = pd.to_datetime(df['Case.Date.Start'])
df['Case.Date.End'] = pd.to_datetime(df['Case.Date.End'])

# Convert Treatment.Start and Treatment.End to datetime
intervention['Treatment.Start'] = pd.to_datetime(intervention['Treatment.Start'])
intervention['Treatment.End'] = pd.to_datetime(intervention['Treatment.End'])

# Function to check if treatment interval overlaps with case interval
def check_overlap(row, treatment_df):
    patient = row['Case.Patient']
    start = row['Case.Date.Start']
    end = row['Case.Date.End']
    treatment_intervals = treatment_df[treatment_df['Case.Patient'] == patient]
    for _, treatment in treatment_intervals.iterrows():
        if (treatment['Treatment.Start'] <= end) and (treatment['Treatment.End'] >= start):
            return True
    return False

# Apply the function to each row in df to create the indicator column
df['Treatment.Indicator'] = df.apply(lambda row: check_overlap(row, intervention), axis=1)

df

## Group intervals


In [ ]:
# In order to track the same total tumor volume during treatment and non-treatment interval we create group ids

# Sort by Case.Patient and Case.Date.Start
df = df.sort_values(by=['Case.Patient', 'Case.Date.Start']).reset_index(drop=True)

# Initialize group ID column
df['Group.ID'] = 0
current_group_id = 1

# Iterate through the DataFrame to assign group IDs
for i in range(len(df)):
    if i == 0:
        df.loc[i, 'Group.ID'] = current_group_id
    else:
        if df.loc[i, 'Case.Patient'] != df.loc[i - 1, 'Case.Patient']:
            # New patient, start new group
            current_group_id += 1
            df.loc[i, 'Group.ID'] = current_group_id
        elif df.loc[i, 'Case.Date.Start'] > df.loc[i - 1, 'Case.Date.End']:
            # Temporal gap, start new group
            current_group_id += 1
            df.loc[i, 'Group.ID'] = current_group_id
        elif df.loc[i, 'Treatment.Indicator'] != df.loc[i - 1, 'Treatment.Indicator']:
            # Change in treatment status, start new group
            current_group_id += 1
            df.loc[i, 'Group.ID'] = current_group_id
        else:
            # Continue the current group
            df.loc[i, 'Group.ID'] = current_group_id

df

## Filter on Non-Treatment vs Treatment Interval

In [ ]:
dfnotreatment =  df[df['Treatment.Indicator'] == False]
dfnotreatment = dfnotreatment.drop(columns=['Volume.Range.Start', 'Volume.Range.End', 'Treatment.Indicator'])
dfnotreatment

In [ ]:
dftreatment =  df[(df['Treatment.Indicator'] == True)]
dftreatment = dftreatment.drop(columns=['Volume.Range.Start', 'Volume.Range.End', 'Treatment.Indicator'])
dftreatment

## Ensuring Continuity of Treatment Periods in Patient Data

### No treatment period

In [ ]:
df = pd.DataFrame(dfnotreatment)
df['Case.Date.Start'] = pd.to_datetime(df['Case.Date.Start'])
df['Case.Date.End'] = pd.to_datetime(df['Case.Date.End'])

# Find missing end dates and add them as new rows ensuring continuity
new_rows = []

for group_id in df['Group.ID'].unique():
    group_df = df[df['Group.ID'] == group_id]
    end_dates = group_df['Case.Date.End'].tolist()
    start_dates = group_df['Case.Date.Start'].tolist()
    
    for end_date in end_dates:
        if end_date not in start_dates:
            volume_end = group_df[group_df['Case.Date.End'] == end_date]['Volume.Mean.End'].values[0]
            new_row = {
                'Case.Date.Start': end_date,
                'Case.Date.End': pd.NaT,  # No end date for the new row
                'Volume.Mean.Start': volume_end,
                'Volume.Mean.End': pd.NaT,  # No volume end for the new row
                'Group.ID': group_id
            }
            new_rows.append(new_row)

# Append new rows to the original DataFrame using pd.concat
new_rows_df = pd.DataFrame(new_rows)
df = pd.concat([df, new_rows_df], ignore_index=True)

# Sort the DataFrame by Group.ID and Start Date
dfnt = df.sort_values(by=['Group.ID', 'Case.Date.Start']).reset_index(drop=True)

dfnt

In [ ]:
final_dfnotreatment = dfnt[['Group.ID', 'Case.Date.Start', 'Volume.Mean.Start']]
final_dfnotreatment

### Convert to CSV

In [ ]:
#This csv file will be used in R for the 'tumgr' library
final_dfnotreatment.to_csv('grouped_patients_nt_interval_v3.csv', index=False)

### Treatment period

In [ ]:
df = pd.DataFrame(dftreatment)
df['Case.Date.Start'] = pd.to_datetime(df['Case.Date.Start'])
df['Case.Date.End'] = pd.to_datetime(df['Case.Date.End'])

# Find missing end dates and add them as new rows
new_rows = []

for group_id in df['Group.ID'].unique():
    group_df = df[df['Group.ID'] == group_id]
    end_dates = group_df['Case.Date.End'].tolist()
    start_dates = group_df['Case.Date.Start'].tolist()
    
    for end_date in end_dates:
        if end_date not in start_dates:
            volume_end = group_df[group_df['Case.Date.End'] == end_date]['Volume.Mean.End'].values[0]
            new_row = {
                'Case.Date.Start': end_date,
                'Case.Date.End': pd.NaT,  # No end date for the new row
                'Volume.Mean.Start': volume_end,
                'Volume.Mean.End': pd.NaT,  # No volume end for the new row
                'Group.ID': group_id
            }
            new_rows.append(new_row)

# Append new rows to the original DataFrame using pd.concat
new_rows_df = pd.DataFrame(new_rows)
df = pd.concat([df, new_rows_df], ignore_index=True)

# Sort the DataFrame by Group.ID and Start Date
dft = df.sort_values(by=['Group.ID', 'Case.Date.Start']).reset_index(drop=True)

dft

In [ ]:
final_dftreatment = dft[['Group.ID', 'Case.Date.Start', 'Volume.Mean.Start']]
final_dftreatment

In [ ]:
# This csv file will be used in R for the 'tumgr' library
final_dftreatment.to_csv('grouped_patients_t_interval.csv', index=False)

## Tumor Grades

In [ ]:
# Reading in tumor-grades data from the Tumor sheet data
tumordata = pd.read_excel('W_23_5814_2024-03-06.xlsx', sheet_name='Tumor')
tumorgrades = tumordata[['PATIENTNR', 'TRTU_Morfologie_oms']]

# # use anonymization keys to replace patient IDs
keys = keys.loc[~(keys['Patient ID'] == '20401820xxx187484')]
keys['Patient ID'] = keys['Patient ID'].astype(int)
tumorgrades = pd.merge(tumorgrades, keys, left_on='PATIENTNR', right_on='Patient ID', how='left')
tumorgrades = tumorgrades.drop(columns=['PATIENTNR', 'Patient ID'])
tumorgrades.rename(columns={'Anonymized Name': 'Case.Patient'}, inplace=True)

# Split the morfologie column into 'type' and 'tumorgrade'
tumorgrades[['type', 'tumorgrade']] = tumorgrades['TRTU_Morfologie_oms'].str.split(', ', expand=True)

# Drop the original 'Morfology name' column
tumorgrades.drop(columns=['TRTU_Morfologie_oms'], inplace=True)
tumorgrades

# Growth rates vs Grades - Non-treatment

In [ ]:
# Read in the Analysis data from the 'tumgr' library in R
tumorgrowth = pd.read_csv('results_totalliver_nt.csv')

# remove rows with no 'g'
tumorgrowth = tumorgrowth.dropna(subset=['g'])
tumorgrowth

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style of the visualization
sns.set(style="whitegrid")

# Create a histogram with Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(tumorgrowth['g'], bins=10, kde=True, color='blue', edgecolor='black')

# Adding title and labels
plt.title('Distribution of Tumor Growth Rates', fontsize=16)
plt.xlabel('Tumor Growth Rate', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Show plot
plt.show()

## Validation: Growth Rates merged with Tumor Grades

In [ ]:
growthrates = dfnotreatment[['Case.Patient', 'Group.ID']].merge(tumorgrowth, left_on='Group.ID', right_on='name', how='left')
growthrates = growthrates.drop_duplicates(subset=['Group.ID']).drop(columns=['name', 'N', 'type', 'selectedFit'])
growthrates = growthrates.groupby('Case.Patient')[['g', 'd', 'phi']].mean().reset_index()
growthrates

# # #merging growthrates with the grades
gandgrade = growthrates.merge(tumorgrades[['Case.Patient', 'tumorgrade']], on='Case.Patient', how='left')

gandgrade = gandgrade.groupby('Case.Patient')[['g', 'd', 'phi']].mean().reset_index().merge(tumorgrades[['Case.Patient', 'tumorgrade']], on='Case.Patient', how='left')
# Remove white space from the 'Category' column
gandgrade['tumorgrade'] = gandgrade['tumorgrade'].str.strip()

# Replace values for graad 3
gandgrade['tumorgrade'] = gandgrade['tumorgrade'].replace({'NNO': 'graad 3'})
gandgrade

In [ ]:
# Calculate mean for each group
mean_by_grade = gandgrade.groupby('tumorgrade')['g'].mean()
print(mean_by_grade)

# Setting the style of the visualization
sns.set(style="whitegrid")

# custom color palette
palette = {
    'graad 1': 'green',
    'graad 2': 'orange',
    'graad 3': 'red'
}

# Creating a histogram with Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(
    gandgrade, 
    x='g', 
    hue='tumorgrade', 
    kde=True, 
    multiple='stack', 
    bins=10, 
    edgecolor='black', 
    palette=palette
)

# Adding title and labels
plt.title('Distribution of Tumor Growth Rates total volume non-treatment interval', fontsize=16)
plt.xlabel('Tumor Growth Rate', fontsize=14)
plt.ylabel('Number of patients', fontsize=14)

plt.show()

In [ ]:
# custom color palette
palette = {
    'graad 1': 'green',
    'graad 2': 'orange',
    'graad 3': 'red'
}

# Creating the boxplot
sns.boxplot(data=gandgrade, x='tumorgrade', y='g', palette=palette)

# Adding title and labels
plt.title('Distribution of Tumor Growth total volume non-treatment interval', fontsize=16)
plt.xlabel('Tumor Grade', fontsize=14)
plt.ylabel('Tumor Growth Rate', fontsize=14)

# Growth rates vs Grades - Treatment Interval

In [ ]:
# Read in the Analysis data from the 'tumgr' library in R
tumorgrowth = pd.read_csv('results_patients_t.csv')
# tumorgrowth = tumorgrowth[(tumorgrowth['d'].isna()) & (tumorgrowth['phi'].isna())]
# tumorgrowth = tumorgrowth.dropna(subset=['g'])
tumorgrowth.shape

In [ ]:
growthrates = dftreatment[['Case.Patient', 'Group.ID']].merge(tumorgrowth, left_on='Group.ID', right_on='name', how='left')
growthrates = growthrates.drop_duplicates(subset=['Group.ID']).drop(columns=['name', 'N', 'type', 'selectedFit'])
growthrates = growthrates.groupby('Case.Patient')[['g', 'd', 'phi']].mean().reset_index()
# #merging growthrates with the grades
gandgrade = growthrates.merge(tumorgrades[['Case.Patient', 'tumorgrade']], on='Case.Patient', how='left')
# Remove white space from the 'Category' column
gandgrade['tumorgrade'] = gandgrade['tumorgrade'].str.strip()
# Replace values for graad 3
gandgrade['tumorgrade'] = gandgrade['tumorgrade'].replace({'NNO': 'graad 3'})
gandgrade

In [ ]:
# Calculate mean growth for each group
mean_growth_by_grade = gandgrade.groupby('tumorgrade')['g'].mean()
print(mean_growth_by_grade)

# Set the style of the visualization
sns.set(style="whitegrid")

# Custom color palette
palette = {
    'graad 1': 'green',
    'graad 2': 'orange',
    'graad 3': 'red'
}

# Create a histogram with Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(
    gandgrade, 
    x='g', 
    hue='tumorgrade', 
    kde=True, 
    multiple='stack', 
    bins=10, 
    edgecolor='black', 
    palette=palette
)

# Adding title and labels
plt.title('Distribution of Tumor Growth Rates total volume treatment interval', fontsize=16)
plt.xlabel('Tumor Growth Rate', fontsize=14)
plt.ylabel('Number of patients', fontsize=14)

plt.show()


In [ ]:
palette = {
    'graad 1': 'green',
    'graad 2': 'orange',
    'graad 3': 'red'
}

# Create the boxplot
sns.boxplot(data=gandgrade, x='tumorgrade', y='g', palette=palette)

# Adding title and labels
plt.title('Distribution of Tumor Growth total volume treatment interval', fontsize=16)
plt.xlabel('Tumor Grade', fontsize=14)
plt.ylabel('Tumor Growth Rate', fontsize=14)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate mean decay for each group
mean_growth_by_grade = gandgrade.groupby('tumorgrade')['d'].mean()
print(mean_growth_by_grade)

# Set the style of the visualization
sns.set(style="whitegrid")

# custom color palette
palette = {
    'graad 1': 'green',
    'graad 2': 'orange',
    'graad 3': 'red'
}

# Create a histogram with Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(
    gandgrade, 
    x='d', 
    hue='tumorgrade', 
    kde=True, 
    multiple='stack', 
    bins=8, 
    edgecolor='black', 
    palette=palette
)

# Adding title and labels
plt.title('Distribution of Tumor decay rates for total volume during treatment interval', fontsize=16)
plt.xlabel('Tumor Growth Rate', fontsize=14)
plt.ylabel('Number of patients', fontsize=14)

# Show plot
plt.show()


# Survival Analysis

In [ ]:
# Reading in the growth rate data
tumorgrowth = pd.read_csv('results_totalliver_nt.csv')
tumorgrowth = tumorgrowth.dropna(subset=['g'])
tumorgrowth

growthrates = dfnotreatment[['Case.Patient', 'Group.ID']].merge(tumorgrowth, left_on='Group.ID', right_on='name', how='left')
growthrates = growthrates.drop_duplicates(subset=['Group.ID']).drop(columns=['name', 'N', 'type', 'selectedFit'])
growthrates = growthrates.groupby('Case.Patient')[['g', 'd', 'phi']].mean().reset_index()

# #merging growthrates with the grades
gandgrade = growthrates.merge(tumorgrades[['Case.Patient', 'tumorgrade']], on='Case.Patient', how='left')
# Remove white space from the 'Category' column
gandgrade['tumorgrade'] = gandgrade['tumorgrade'].str.strip()
# Replace values for graad 3
gandgrade['tumorgrade'] = gandgrade['tumorgrade'].replace({'NNO': 'graad 3'})
gandgrade

### Merging tumor grade data with tumor growth rates

In [ ]:
tumordata = pd.read_excel('W_23_5814_2024-03-06.xlsx', sheet_name='Tumor')
# we are interested in the date of diagnosis and last follow up or date or last scan date
tumorgrades = tumordata[['PATIENTNR', 'Date_diagnosis (Stefano)', 'HiX_Date_of_death', 'TRTU_Laatste_follow_up_datum'
,'TRTU_Patientstatus_bij_laatste_follow_up_oms']]
tumorgrades['overleden'] = tumorgrades['TRTU_Patientstatus_bij_laatste_follow_up_oms'].str.contains('overleden', case=False, na=False)

# # use anonymization keys to replace patient IDs
keys = keys.loc[~(keys['Patient ID'] == '20401820xxx187484')]
keys['Patient ID'] = keys['Patient ID'].astype(int)
tumorgrades = pd.merge(tumorgrades, keys, left_on='PATIENTNR', right_on='Patient ID', how='left')
tumorgrades = tumorgrades.drop(columns=['PATIENTNR', 'Patient ID'])
tumorgrades.rename(columns={'Anonymized Name': 'Case.Patient'}, inplace=True)

# Drop the original 'full_name' column if you no longer need it
# tumorgrades.drop(columns=['TRTU_Morfologie_oms'], inplace=True)
gradesdeath = gandgrade.merge(tumorgrades, on='Case.Patient')
gradesdeath = gradesdeath.merge(last_check_up_dates, on='Case.Patient')
gradesdeath['end_date'] = gradesdeath['HiX_Date_of_death'].combine_first(gradesdeath['TRTU_Laatste_follow_up_datum'])
gradesdeath['end_date'] = gradesdeath['end_date'].combine_first(gradesdeath['Case.Date'])
gradesdeath

In [ ]:
# Checking if there is a patient with no end date
gradesdeath[gradesdeath['end_date'].isna() == True]

### Kaplan Meier Fitter + validation using pairwise log rank test

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.statistics import pairwise_logrank_test

# Prepare data for Kaplan-Meier estimator for each treatment group
kmf = KaplanMeierFitter()

# Define color mapping for each grade
color_map = {'graad 1': 'green', 'graad 2': 'orange', 'graad 3': 'red'}

# Plot the Kaplan-Meier curves
for treatment_group in gradesdeath['tumorgrade'].unique():
    treatment_data = gradesdeath[gradesdeath['tumorgrade'] == treatment_group]
    print("Treatment Group:", treatment_group)
    print("Data Length:", len(treatment_data))
    
    if len(treatment_data) > 0:
        durations = (treatment_data['end_date'] - treatment_data['Date_diagnosis (Stefano)']).dt.days
        event_observed = treatment_data['overleden']
        
        kmf.fit(durations=durations, event_observed=event_observed, label=treatment_group)
        kmf.plot(color=color_map[treatment_group])
    else:
        print("No data available for this treatment group.")

# Plot settings
plt.title('Kaplan-Meier Overall Survival Curve by Tumor Grade')
plt.xlabel('Time (days)')
plt.ylabel('Survival Probability')
plt.grid(True)
plt.legend(title='Tumor Grade')
plt.show()

# Prepare data for pairwise log-rank test
durations = (gradesdeath['end_date'] - gradesdeath['Date_diagnosis (Stefano)']).dt.days
event_observed = gradesdeath['overleden']
groups = gradesdeath['tumorgrade'].astype(str)  # Ensure all entries are strings

# Perform pairwise log-rank test
results = pairwise_logrank_test(durations, groups, event_observed)
print("Pairwise Log-Rank Test Results:")
results.print_summary()
